# TSPTW by OR-Tools

In [1]:
import random as rd
import numpy as np
import pandas as pd
import math
import copy as copy
from ortools.constraint_solver import routing_enums_pb2 
from ortools.constraint_solver import pywrapcp 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
def create_data_model():
    data = {}
    data["locations"] = [(round(np.random.uniform(0, MAX_X),2), round(np.random.uniform(0, MAX_Y),2)) for i in range(NUM_NODES)]
    data["locations"][0] = (MAX_X / 2, MAX_Y / 2)
    data["distance_matrix"] = compute_distance_matrix(data["locations"])
    data["time_matrix"] = compute_time_matrix(data["distance_matrix"])
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data

def compute_distance_matrix(nodes):
    distance_matrix = []
    for node_i in nodes:
        distances = []
        for node_j in nodes:
            if node_i == node_j:
                distances.append(0)
            else:
                manhattan_distance = np.sum(np.abs(np.array(node_i) - np.array(node_j)))
                distances.append(round(manhattan_distance, 2))
        distance_matrix.append(distances)
    return distance_matrix

def compute_time_matrix(distance_matrix):
    time_matrix = []
    for i in distance_matrix:
        time_matrix.append(list(map(lambda x: int(x * 60), i)))
    return time_matrix

In [3]:
def main():

    manager = pywrapcp.RoutingIndexManager(len(data["time_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        0,  # allow waiting time
        99999999,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.time_limit.seconds = 10
    search_parameters.solution_limit = 100

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_second = print_solution(data, manager, routing, solution)
        cumul_data = get_cumul_data(solution, routing, time_dimension)
        return total_second
    else:
        print('No solution.')

def get_cumul_data(solution, routing, dimension):
    cumul_data = []
    for route_nbr in range(routing.vehicles()):
        route_data = []
        index = routing.Start(route_nbr)
        dim_var = dimension.CumulVar(index)
        route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            dim_var = dimension.CumulVar(index)
            route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
            cumul_data.append(route_data)
    return cumul_data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}sec\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}s")
    print(f"Total serivce time: {(NUM_NODES - 1) * 120}s")
    return (total_time + ((NUM_NODES - 1) * 120))
    

In [28]:
result = { 
    "size": [], 
    "num_node": [], 
    "total_time": []
    }

MAX_X, MAX_Y = (25, 25)

for i in range(10, 250, 10):

    NUM_NODES = i

    for j in range(10):

        result["size"].append((MAX_X, MAX_Y))
        result["num_node"].append(i)
        data = create_data_model()
        travel_time = main()
        serivce_time = (NUM_NODES - 1) * 120
        result["total_time"].append(round(travel_time / 60, 2))
        print('-Service Time: %.2f min' %(serivce_time / 60))
        print('-Total Time: %.2f min' %(travel_time / 60))

Objective: 5484
Route for vehicle 0:
0 Time(0,0) -> 8 Time(361,361) -> 9 Time(1032,1032) -> 2 Time(1626,1626) -> 6 Time(1830,1830) -> 4 Time(2154,2154) -> 3 Time(2399,2399) -> 7 Time(3270,3270) -> 5 Time(3513,3513) -> 1 Time(4568,4568) -> 0 Time(5484,5484)
Time of the route: 5484sec

Total time of all routes: 5484s
Total serivce time: 1080s
-Service Time: 18.00 min
-Total Time: 109.40 min
Objective: 5739
Route for vehicle 0:
0 Time(0,0) -> 3 Time(104,104) -> 5 Time(854,854) -> 7 Time(1446,1446) -> 2 Time(2580,2580) -> 8 Time(3001,3001) -> 6 Time(3442,3442) -> 4 Time(3773,3773) -> 1 Time(4063,4063) -> 9 Time(5037,5037) -> 0 Time(5739,5739)
Time of the route: 5739sec

Total time of all routes: 5739s
Total serivce time: 1080s
-Service Time: 18.00 min
-Total Time: 113.65 min
Objective: 6167
Route for vehicle 0:
0 Time(0,0) -> 1 Time(228,228) -> 9 Time(747,747) -> 7 Time(1535,1535) -> 2 Time(2888,2888) -> 6 Time(3885,3885) -> 5 Time(4562,4562) -> 3 Time(5028,5028) -> 4 Time(5478,5478) -> 8 

In [29]:
df_result = pd.DataFrame(result)
df_result

,size,num_node,total_time
0,"(25, 25)",10,109.40
1,"(25, 25)",10,113.65
2,"(25, 25)",10,120.78
3,"(25, 25)",10,106.00
4,"(25, 25)",10,96.95
...,...,...,...
235,"(25, 25)",240,858.65
236,"(25, 25)",240,860.88
237,"(25, 25)",240,863.77
238,"(25, 25)",240,863.33


In [30]:
df_result.to_csv(f'{MAX_X}x{MAX_Y}_time.csv')

In [31]:
round(df_result.groupby('num_node')['total_time'].mean() / 60, 2)

num_node
10      1.82
20      2.56
30      3.40
40      4.02
50      4.65
60      5.33
70      6.00
80      6.48
90      6.88
100     7.50
110     8.07
120     8.51
130     9.05
140     9.56
150    10.08
160    10.55
170    11.01
180    11.52
190    11.99
200    12.58
210    13.08
220    13.36
230    13.84
240    14.39
Name: total_time, dtype: float64